In [1]:
from bs4 import BeautifulSoup
import mechanicalsoup
import requests
import time, os
from selenium import webdriver
import math
import pickle
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tummee.com/login?")

cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://www.tummee.com/sequences/teachers/daily")

In [6]:
f = open("iyengar_urls", "rb")
iyengar_urls = pickle.load(f)
f.close()

In [4]:
def get_poses(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        title = re.sub(r"[\t\n0-9.]", "", soup.find("h1").text.strip())
        find_poses = soup.find_all("a", class_="english-title")
        poses = [re.sub(r"[\t\n0-9.]", "", i.text.strip()) for i in find_poses]
        class_type = "Iyengar"
    except AttributeError:
        info = {"Title": np.nan}
    else: 
        info = {"Title": title, "Poses": poses, "Class Type": class_type}
    return info

In [7]:
iyengar_classes = [get_poses(url) for url in iyengar_urls]

In [9]:
df = pd.DataFrame(iyengar_classes)
df

,Title,Poses,Class Type
0,←Standing Arms,"[Arms Raised Overhead Palms Front Close Up, Ar...",Iyengar
1,←Gayatri Flow - original de Diana Vaz revisto ...,"[Triangle Pose, Half Moon Pose, Warrior Pose I...",Iyengar
2,←Core,"[Cobra Dance Flow, Standing Squat Pose Knee Mo...",Iyengar
3,←Secuencia PIES,"[Virasana sobre ladrillo, Trabajo de tobillos,...",Iyengar
4,←Iyengar Course - Week -,"[Mountain Pose, Five Pointed Star Pose, Triang...",Iyengar
...,...,...,...
1206,←Обичай себе си - седмица,"[Seated Mountain Pose, Easy Pose Variation Arm...",Iyengar
1207,← joint movement,"[Neck Rotation And Massage, Arms Rotations, St...",Iyengar
1208,←Level I - Stehhaltungen,"[Samasthiti, Ūrdhva Hastāsana, Ūrdhva Baddhāṅg...",Iyengar
1209,NaN,NaN,NaN


In [11]:
f = open("iyengar_df", "wb")
pickle.dump(df, f)
f.close()